In [18]:
import numpy as np
import cv2
import json
from pathlib import Path
from PIL import Image, ImageDraw
import math

def split_and_label_small_defects(binary_mask, min_area=1, max_diameter=7):
    """
    输入：
      - binary_mask: uint8 二值图 (255 表示候选缺陷像素, 0 背景)
      - min_area: 连通块最小面积，低于此丢弃
      - max_diameter: 最终允许的小圆最大直径
    输出：
      - defect_circles: [[cx, cy], [cx+r, cy]] 列表，每个圆直径 ≤ max_diameter
    """
    defect_circles = []
    num_labels, labels, stats, _ = cv2.connectedComponentsWithStats(binary_mask)
    for lbl in range(1, num_labels):
        area = stats[lbl, cv2.CC_STAT_AREA]
        if area < min_area:
            continue

        single_mask = (labels == lbl).astype(np.uint8) * 255
        cnts, _ = cv2.findContours(single_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        if not cnts:
            continue
        cnt = cnts[0]
        (cx, cy), r = cv2.minEnclosingCircle(cnt)
        diameter = 2 * r

        if diameter <= max_diameter:
            defect_circles.append([[float(cx), float(cy)], [float(cx + r), float(cy)]])
            continue

        # 拆分：距离变换 + 分水岭
        single_bin = single_mask.copy()
        single_bin[single_bin > 0] = 1
        dist = cv2.distanceTransform(single_bin, distanceType=cv2.DIST_L2, maskSize=5)
        _, sure_fg = cv2.threshold(dist, dist.max() * 0.6, 255, 0)
        sure_fg = np.uint8(sure_fg)
        num_fg, markers = cv2.connectedComponents(sure_fg)
        markers = markers + 1
        markers[single_bin == 0] = 0

        single_color = cv2.cvtColor(single_mask, cv2.COLOR_GRAY2BGR)
        cv2.watershed(single_color, markers)

        for marker_id in range(2, num_fg + 2):
            sub_region = np.uint8(markers == marker_id)
            if sub_region.sum() < min_area:
                continue
            sub_cnts, _ = cv2.findContours(sub_region, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
            if not sub_cnts:
                continue
            sc = sub_cnts[0]
            (scx, scy), sr = cv2.minEnclosingCircle(sc)
            sdiam = 2 * sr
            if sdiam > max_diameter:
                sr = max_diameter / 2.0
            defect_circles.append([[float(scx), float(scy)], [float(scx + sr), float(scy)]])
    return defect_circles

def circle_intersection_area(c1, c2):
    x0, y0, r0 = c1
    x1, y1, r1 = c2
    d = math.hypot(x1 - x0, y1 - y0)
    if d >= r0 + r1:
        return 0.0
    if d <= abs(r0 - r1):
        return math.pi * min(r0, r1)**2

    # 下面是两圆部分重叠时的计算
    r0_sq, r1_sq, d_sq = r0**2, r1**2, d**2
    alpha = math.acos((d_sq + r0_sq - r1_sq) / (2*d*r0)) * 2
    beta  = math.acos((d_sq + r1_sq - r0_sq) / (2*d*r1)) * 2
    area0 = 0.5 * r0_sq * (alpha - math.sin(alpha))
    area1 = 0.5 * r1_sq * (beta  - math.sin(beta))
    return area0 + area1

tif_dir  = Path(r'D:\Study\Postgraduate\S2\Project\Code\Resource\Original\1_2_3_4\1')
json_dir = Path(r'D:\Study\Postgraduate\S2\Project\Code\Resource\Middle Stage B\1_2_3_4\1')
output_dir = Path(r'D:\Study\Postgraduate\S2\Project\Code\Resource\Test')
output_dir.mkdir(parents=True, exist_ok=True)

brightness_threshold = 255  # 根据需要调整

# 获取所有 tif
fnames = [p for p in tif_dir.iterdir() if p.suffix.lower() == '.tif']
for image_path in fnames:
    # 对应 JSON 名称
    json_filename = f"{image_path.stem}_mainbody.json"
    json_path = json_dir / json_filename
    if not json_path.exists():
        print(f"Skip, not found: {json_filename}")
        continue

    # 读取原始标注
    with json_path.open('r', encoding='utf-8') as f:
        ann = json.load(f)

    img = Image.open(image_path).convert("RGB")
    w, h = img.size

    # 生成主体掩码
    mask_mb = np.zeros((h, w), dtype=np.uint8)
    for s in ann["shapes"]:
        if s["label"].lower() == "main body":
            tmp = Image.new("L", (w, h), 0)
            draw = ImageDraw.Draw(tmp)
            pts = [tuple(p) for p in s["points"]]
            draw.polygon(pts, fill=1)
            mask_mb[np.array(tmp, bool)] = 1

    mb_uint8 = (mask_mb * 255).astype(np.uint8)
    dist_to_border = cv2.distanceTransform(mb_uint8, distanceType=cv2.DIST_L2, maskSize=5)
    gray = cv2.imread(str(image_path), cv2.IMREAD_GRAYSCALE)

    # 腐蚀去边缘
    erode_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (7, 7))
    inner_mask = cv2.erode(mask_mb, erode_kernel, iterations=1)
    gray_masked = gray.copy()
    gray_masked[inner_mask != 1] = 255

    # 分块自适应阈值
    bw_combined = np.zeros((h, w), dtype=np.uint8)
    x_quarters = [0, w//4, 2*w//4, 3*w//4, w]
    y_quarters = [0, h//4, 2*h//4, 3*h//4, h]
    for i in range(4):
        for j in range(4):
            x0, x1 = x_quarters[i], x_quarters[i+1]
            y0, y1 = y_quarters[j], y_quarters[j+1]
            sub_mask = np.zeros((h, w), dtype=np.uint8)
            sub_mask[y0:y1, x0:x1] = 1
            mb_sub = (inner_mask==1) & (sub_mask==1)
            pixels = gray[mb_sub]
            if pixels.size == 0:
                continue
            median_val = float(np.median(pixels))
            thresh_val = int(median_val * 0.925)
            bw = np.zeros((h, w), dtype=np.uint8)
            bw[(mb_sub) & (gray_masked < thresh_val)] = 255
            bw_combined = cv2.bitwise_or(bw_combined, bw)

    # 连通组件过滤
    num_labels, labels, stats, _ = cv2.connectedComponentsWithStats(bw_combined)
    mask_defect = np.zeros((h, w), dtype=np.uint8)
    for k in range(1, num_labels):
        if stats[k, cv2.CC_STAT_AREA] >= 1:
            mask_defect[labels==k] = 255

    # 查找轮廓并拟圆
    contours, _ = cv2.findContours(mask_defect, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    defect_circles = []
    for cnt in contours:
        if cv2.contourArea(cnt) < 1:
            continue
        (cx, cy), r = cv2.minEnclosingCircle(cnt)
        diameter = 2 * r
        cx_i, cy_i = int(round(cx)), int(round(cy))
        dist_center = dist_to_border[cy_i, cx_i]
        if diameter <= 9:
            mask_c = np.zeros((h, w), dtype=np.uint8)
            cv2.circle(mask_c, (cx_i, cy_i), int(round(r)), 1, -1)
            vals = gray[mask_c==1]
            if vals.size>0 and vals.min()<=brightness_threshold:
                defect_circles.append([[float(cx), float(cy)], [float(cx+r), float(cy)]])
        else:
            if dist_center < 15:
                continue
            single = np.zeros((h, w), dtype=np.uint8)
            cv2.drawContours(single, [cnt], -1, 255, -1)
            splits = split_and_label_small_defects(single, min_area=2, max_diameter=9
            )
            for sc in splits:
                scx, scy = int(round(sc[0][0])), int(round(sc[0][1]))
                sr = abs(sc[1][0]-sc[0][0])
                mask_c = np.zeros((h, w), dtype=np.uint8)
                cv2.circle(mask_c, (scx, scy), int(round(sr)), 1, -1)
                vals = gray[mask_c==1]
                if vals.size>0 and vals.min()<=brightness_threshold:
                    defect_circles.append(sc)

    # 去重重叠圆
    circles_params = [(c[0][0], c[0][1], abs(c[1][0]-c[0][0])) for c in defect_circles]
    keep = [True]*len(circles_params)
    for i in range(len(circles_params)):
        if not keep[i]: continue
        for j in range(i+1, len(circles_params)):
            if not keep[j]: continue
            ai = math.pi*circles_params[i][2]**2
            aj = math.pi*circles_params[j][2]**2
            inter = circle_intersection_area(circles_params[i], circles_params[j])
            if inter and inter/min(ai, aj)>=0.1:
                keep[j] = False
    filtered = [defect_circles[k] for k in range(len(defect_circles)) if keep[k]]

    # 构建新 JSON，保证 imagePath 指向 tif_dir 中的源文件
    new_shapes = [s for s in ann["shapes"] if s["label"].lower()=="main body"]
    for pts in filtered:
        new_shapes.append({
            "label": "defect",
            "points": pts,
            "group_id": None,
            "shape_type": "circle",
            "flags": {}
        })
    labelme = {
        "version": ann.get("version", "5.2.1"),
        "flags": ann.get("flags", {}),
        # 关键修改：用绝对 tif 路径，确保打开时定位正确
        "imagePath": str(image_path),
        "imageHeight": h,
        "imageWidth": w,
        # 不嵌入 imageData，留空
        "imageData": None,
        "shapes": new_shapes
    }

    # 写出 JSON
    out_path = output_dir / image_path.with_suffix('.json').name
    with out_path.open('w', encoding='utf-8') as f:
        json.dump(labelme, f, ensure_ascii=False, indent=2)
    print(f"✓ Saved JSON: {out_path.name}")

    # 保存含缺陷全图掩码（0 背景，1 主体，2 缺陷）
    full_mask = mask_mb.copy()
    for pts in filtered:
        cx, cy = int(round(pts[0][0])), int(round(pts[0][1]))
        r = abs(pts[1][0]-pts[0][0])
        cv2.circle(full_mask, (cx, cy), int(round(r)), 2, -1)
    mask_path = output_dir / f"{image_path.stem}.npy"
    np.save(str(mask_path), full_mask)
    print(f"✓ Saved mask: {mask_path.name}")


✓ Saved JSON: crop_Componenets_1.transformed059.json
✓ Saved mask: crop_Componenets_1.transformed059.npy
✓ Saved JSON: crop_Componenets_1.transformed060.json
✓ Saved mask: crop_Componenets_1.transformed060.npy
✓ Saved JSON: crop_Componenets_1.transformed061.json
✓ Saved mask: crop_Componenets_1.transformed061.npy
✓ Saved JSON: crop_Componenets_1.transformed062.json
✓ Saved mask: crop_Componenets_1.transformed062.npy
✓ Saved JSON: crop_Componenets_1.transformed063.json
✓ Saved mask: crop_Componenets_1.transformed063.npy
✓ Saved JSON: crop_Componenets_1.transformed064.json
✓ Saved mask: crop_Componenets_1.transformed064.npy
✓ Saved JSON: crop_Componenets_1.transformed065.json
✓ Saved mask: crop_Componenets_1.transformed065.npy
✓ Saved JSON: crop_Componenets_1.transformed066.json
✓ Saved mask: crop_Componenets_1.transformed066.npy
✓ Saved JSON: crop_Componenets_1.transformed067.json
✓ Saved mask: crop_Componenets_1.transformed067.npy
✓ Saved JSON: crop_Componenets_1.transformed068.json
✓ 